Rag From Scratch: Query Transformations
Query transformations are a set of approaches focused on re-writing and / or modifying questions for retrieval.



In [6]:
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_community.embeddings import DashScopeEmbeddings 

from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import bs4
from langchain import hub
import os
from langchain.chat_models import init_chat_model

from dotenv import load_dotenv
import os
load_dotenv()
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'

Part 5: Multi Query

In [7]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)
# Make splits
splits = text_splitter.split_documents(blog_docs)


embeding = DashScopeEmbeddings(model="text-embedding-v1")


vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=embeding)


retriever = vectorstore.as_retriever()








##  promote

In [9]:
from langchain.prompts import ChatPromptTemplate

# Multi Query: Different Perspectives
template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

from langchain_core.output_parsers import StrOutputParser

llm = init_chat_model(
    model="deepseek-chat", 
    model_provider="deepseek",
    api_key=os.getenv('DEEPSEEK_API_KEY')  # 添加这行
)

generate_queries = (
    prompt_perspectives 
    | llm 
    | StrOutputParser()   #  3. 解析输出为字符串
    | (lambda x: x.split("\n"))
)

In [10]:
from langchain.load import dumps, loads
# dumps: 将 LangChain Document 对象序列化为字符串
# loads: 将字符串反序列化回 Document 对象
# 得到 retrieval_results（多个文档列表的列表）
# 假设有3个查询变体，每个返回不同的文档
# retrieval_results = [
#     [doc1, doc2, doc3],  # 查询1的结果
#     [doc2, doc4, doc5],  # 查询2的结果  
#     [doc1, doc6]         # 查询3的结果
# ]

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]


# Retrieval
question = "What is task decomposition for LLM agents?"

retrieval_chain = generate_queries | retriever.map() | get_unique_union

# # 不使用 .map() 的情况
# retriever.invoke("单个查询")  # 返回：[doc1, doc2, doc3]

# # 使用 .map() 的情况
# retriever.map().invoke(["查询1", "查询2", "查询3"])
# # 返回：[[doc1, doc2], [doc3, doc4], [doc5, doc6]]

# retrieval_chain = (
#     generate_queries           # 输入：单个问题
#     | retriever.map()         # 输出：多个查询 → 多个文档列表
#     | get_unique_union        # 输入：多个文档列表 → 输出：去重后的单个文档列表
# )

docs = retrieval_chain.invoke({"question": question})
len(docs)










C:\Users\Administrator\AppData\Local\Temp\ipykernel_108584\4144763748.py:19: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


7

In [11]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = init_chat_model(
    model="deepseek-chat", 
    model_provider="deepseek",
    api_key=os.getenv('DEEPSEEK_API_KEY')  # 添加这行
)
final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)
# 总结：
# "question"：传递字符串字面量 "question"
# itemgetter("question")：传递一个函数，用于从输入数据中动态提取 question 字段的值
# 目的：确保传递的是用户的实际问题内容，而不是字段名
# 这是 LangChain 管道设计中处理动态数据提取的标准模式。
final_rag_chain.invoke({"question":question})

'Task decomposition for LLM (Large Language Model) agents refers to the process of breaking down complex tasks into smaller, more manageable subgoals or steps. This enables the agent to handle intricate tasks more efficiently by addressing them in a structured manner. \n\nKey aspects of task decomposition include:\n1. **Chain of Thought (CoT)**: A prompting technique where the model is instructed to "think step by step," decomposing hard tasks into simpler steps. This approach helps in interpreting the model\'s reasoning process.\n   \n2. **Tree of Thoughts (ToT)**: An extension of CoT that explores multiple reasoning possibilities at each step, creating a tree-like structure of thoughts. This allows for more comprehensive exploration of solutions, using methods like breadth-first search (BFS) or depth-first search (DFS).\n\n3. **Methods of Decomposition**:\n   - **Simple prompting**: Using instructions like "Steps for XYZ.\\\\n1." or "What are the subgoals for achieving XYZ?"\n   - **

Part 6: RAG-Fusion

In [12]:
from langchain.prompts import ChatPromptTemplate

template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)


In [ ]:
llm = init_chat_model(
    model="deepseek-chat", 
    model_provider="deepseek",
    api_key=os.getenv('DEEPSEEK_API_KEY')  # 添加这行
)
generate_queries = (
    prompt_rag_fusion
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)
# # LLM 生成的回答通常是这样的格式：
# response = """
# 什么是机器学习？
# 机器学习的定义是什么？
# 如何理解机器学习？
# 机器学习的基本概念
# 机器学习是什么技术？
# """

# 最终生成多个查询如：
# ["什么是机器学习？", "机器学习的定义是什么？", "如何理解机器学习？", "机器学习的基本概念", "机器学习是什么技术？"]









In [13]:
# Reciprocal_rank_fusion算法
from langchain.load import dumps, loads
def reciprocal_rank_fusion(results: list[list], k=60):
    """
    对已经排好序的多个文档列表[[doc1, doc2, doc3], [doc4, doc5, doc6], [doc7, doc8, doc9]]
    进行融合排序，计算分数
    """
    fusion_results = {}

    for docs in results:
        for i, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fusion_results:
                fusion_results[doc_str] = 0
            
            fusion_results[doc_str] += 1 / (i + k)
    # 对fusion_results进行排序
    sorted_results = sorted(fusion_results.items(), key=lambda x: x[1], reverse=True)
    reanked_results = [(loads(doc),score) for doc,score in sorted_results]
    return reanked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() |reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)


9

rag

In [ ]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, "question": itemgetter("question")}
    | prompt
    | llm
    | StrOutputParser()
)
final_rag_chain.invoke({"question":question})












Part 7: Decomposition

In [15]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [16]:
llm = init_chat_model(
    model="deepseek-chat", 
    model_provider="deepseek",
    api_key=os.getenv('DEEPSEEK_API_KEY')  # 添加这行
)

# chain
generate_queries_decomposition = (
    prompt_decomposition
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)
# Run
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question":question})

In [17]:
questions

['Here are three sub-questions related to the main components of an LLM-powered autonomous agent system:  ',
 '',
 '1. **What are the key architectural components of an LLM-powered autonomous agent?**  ',
 '2. **How does memory and context management work in an LLM-based autonomous agent?**  ',
 '3. **What role do APIs and external tools play in an LLM autonomous agent system?**  ',
 '',
 'These queries help break down the original question into more specific aspects of the system, such as architecture, memory handling, and integration with external tools.']

HYDE

In [18]:
from langchain.prompts import ChatPromptTemplate

# HyDE document genration
template = """Please write a scientific paper passage to answer the question
Question: {question}
Passage:"""
prompt_hyde = ChatPromptTemplate.from_template(template)

llm = init_chat_model(
    model="deepseek-chat", 
    model_provider="deepseek",
    api_key=os.getenv('DEEPSEEK_API_KEY')  # 添加这行
)

generate_docs_for_retrieval = (
    prompt_hyde
    | llm
    | StrOutputParser()
)

# Run
question = "What is task decomposition for LLM agents?"
generate_docs_for_retrieval.invoke({"question":question})




'**Task Decomposition for LLM Agents**  \n\nTask decomposition is a fundamental capability of Large Language Model (LLM)-based agents, enabling them to break down complex, high-level tasks into smaller, more manageable subtasks. This process enhances the agent\'s ability to plan, reason, and execute tasks efficiently by addressing each component sequentially or in parallel, depending on the task structure.  \n\n### **Mechanisms of Task Decomposition**  \nLLM agents employ various strategies for task decomposition, including:  \n1. **Step-by-Step Breakdown**: The agent recursively divides a task into finer-grained steps, ensuring logical coherence and feasibility. For example, the instruction "Plan a conference" may be decomposed into subtasks such as "Select a venue," "Invite speakers," and "Promote the event."  \n2. **Hierarchical Planning**: The agent organizes tasks in a tree-like structure, where high-level goals branch into subgoals with dependencies. This is particularly useful i

In [19]:
# 检索
retriever_chain = (
    generate_docs_for_retrieval
    | retriever
)

# 运行检索链条
retrieval_results = retriever_chain.invoke({"question": question})


In [20]:
retrieval_results

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Component One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a

RAG

In [ ]:
# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, "question": itemgetter("question")}|
    prompt |
    llm |
    StrOutputParser())

final_rag_chain.invoke({"context":retireved_docs,"question":question})